# Ingest Qualifying Data

In [0]:
dbutils.widgets.text("env", "dev", "Environment")

env = dbutils.widgets.get("env")

In [0]:
%run ../config $env=$env

In [0]:
%run ../utils

In [0]:
csv_file_path = raw_data_folder_path + "qualifying.csv"

df = (
    spark.read.format("csv")
    .option("header", True)
    .option("inferSchema", True)
    .load(csv_file_path)
)

df.display() if env == "dev" else None

In [0]:
import pyspark.sql.functions as F

df_transformed = (
    df
    .withColumnRenamed("constructorId", "constructor_id")
    .withColumnRenamed("driverId", "driver_id")
    .withColumnRenamed("qualifyId", "qualify_id")
    .withColumnRenamed("raceId", "race_id")
    .withColumn("ingestion_date", F.current_timestamp())
)

df_transformed.display() if env == "dev" else None

In [0]:
df_transformed = fillna_str(df_transformed, r"\N")

df_transformed.display()  if env == "dev" else None

In [0]:
df_transformed.write.format("delta").saveAsTable("bronze_tbl_qualifying", mode="overwrite")